In [ ]:
!pip install -q tensorflow

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import os
import shutil
from google.colab import drive

drive.mount('/content/drive')

drive_dataset_path = '/content/drive/MyDrive/DATASET/unzip/Structured'
local_dataset_path = '/content/Structured'

if not os.path.exists(local_dataset_path):
    shutil.copytree(drive_dataset_path, local_dataset_path)

train_path = os.path.join(local_dataset_path, 'train')
test_path = os.path.join(local_dataset_path, 'test')

image_size = 224
batch_size = 32

train_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

test_data = ImageDataGenerator(rescale=1./255)

train_loader = train_data.flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_loader = test_data.flow_from_directory(
    test_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
base_model.trainable = True

features = base_model.output
features = GlobalAveragePooling2D()(features)
features = Dropout(0.5)(features)
features = Dense(128, activation='relu')(features)
features = Dropout(0.3)(features)
output_layer = Dense(train_loader.num_classes, activation='softmax')(features)

model = Model(inputs=base_model.input, outputs=output_layer)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_adjust = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

history = model.fit(
    train_loader,
    validation_data=test_loader,
    epochs=50,
    callbacks=[early_stop, lr_adjust]
)

model.save('/content/efficientnetb0_har_model.h5')


![Alt text](epoch.jpg)

In [2]:


import os
import shutil


drive_dataset_path = r'C:\Users\AKSHAT\Desktop\ECHO_Final\Structured_Image_Dataset'  

local_dataset_path = r'C:\Users\AKSHAT\Desktop\ECHO_Final\Structured_Image_Dataset'  

if not os.path.exists(local_dataset_path):
    if os.path.exists(drive_dataset_path):
        shutil.copytree(drive_dataset_path, local_dataset_path)
    else:
        raise FileNotFoundError(f"Source path not found: {drive_dataset_path}")


from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224
batch_size = 32
test_path = os.path.join(local_dataset_path, 'test')

test_data = ImageDataGenerator(rescale=1./255)
test_loader = test_data.flow_from_directory(
    test_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

from tensorflow.keras.models import load_model

model = load_model('EchoVision_1.h5')

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

pred_probs = model.predict(test_loader)
y_pred = np.argmax(pred_probs, axis=1)
y_true = test_loader.classes
class_labels = list(test_loader.class_indices.keys())


print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))


cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Micro): {f1_micro:.4f}")
print(f"F1 Score (Weighted): {f1_weighted:.4f}")


ModuleNotFoundError: No module named 'tensorflow'